In [36]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from langdetect import detect
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
class Summarize:
    def __init__(self,paragraph):
        self.vec=TfidfVectorizer()
        self.vec.fit_transform([paragraph])
        self.col=self.vec.get_feature_names_out()
        self.paragraph=paragraph
        self.language=detect(self.paragraph)
        if detect(self.language) != "en":
            #stop_words
            self.stop_words=open("stopwords.txt","r",encoding="utf-8")
            self.stop_words=self.stop_words.read()
            self.stop_words=self.stop_words.split("\n")
            
            #num file
            self.nepali_num=open("numbers.txt","r",encoding="utf-8")
            self.nepali_num=self.nepali_num.read()
            self.nepali_num=self.nepali_num.split(",")
            
            #suffix file
            self.nepali_suffix=open("suffix.txt","r",encoding="utf-8")
            self.nepali_suffix=self.nepali_suffix.read()
            self.nepali_suffix=self.nepali_suffix.split("\n")
    
    def nepali_process_paragraph(self):
        paragraph=str(self.paragraph)
        
        #removing \n and \ufeff
        remove=['\n','\ufeff']
        for i in remove:
            paragraph.replace(i,'')
        
        #read stop words
        #Remove Stop Words
        #word_tokens = Tokenizer().word_tokenize(text)
        word_tokens = paragraph.split(" ")
        filtered_list = [w for w in word_tokens if not w in self.stop_words]
        
        #Remove Nepali numbers
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(self.nepali_num)):
                if self.nepali_num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        for filter in num_filter:
            filtered_list.remove(filter)
        
        #Remove English numbers
        num=['0','1','2','3','4','5','6','7','8','9']
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(num)):
                if num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        for filter in num_filter:
            filtered_list.remove(filter)
        stemmed_string=' '.join(filtered_list)
        
        return stemmed_string
            
    def english_process_paragraph(self):
        paragraph=str(self.paragraph)
        #lowercasing
        paragraph=paragraph.lower()
        #Remove Stop Words
        stop_words=set(stopwords.words('english'))
        word_tokens = word_tokenize(paragraph)
        filtered_list = [w for w in word_tokens if not w in stop_words]
        
        
        #Remove numbers and special Symbols
        #words like 100m 2m were not removed so using this
        num=['0','1','2','3','4','5','6','7','8','9']
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(num)):
                if num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        
        for filter in num_filter:
            filtered_list.remove(filter)
                    
        filtered_list = [w for w in filtered_list if w.isalnum()]
        
        #Lematizing
        wordnet_lemmatizer=WordNetLemmatizer()
        lemmatized_list=[wordnet_lemmatizer.lemmatize(w,wordnet.VERB) for w in filtered_list]
        lemmatized_string=' '.join(lemmatized_list)
        
        return lemmatized_string
    
    def calc_idf(self):
        doc_count=len(self.paragraph)
        df={}
        idf={}
        for char in self.col:
            df[char]=0
            idf[char]=0
        #Calculating df
        for i in range(0,len(self.col)):
            for j in range(0,len(self.paragraph)):
                if self.col[i] in self.paragraph[j]:
                    df[self.col[i]]+=1
        #Calculating idf
        for char in self.col:
            idf[char]=math.log((doc_count+1)/(1+df[char]))+1
        return(idf)
    
    def calc_tf_idf(self,sentence):
        idf=self.calc_idf()
        
        tf_idf={}
        word_count={}
        
        for ch in self.col:
            tf_idf[ch]=0
            word_count[ch]=0
        #Calculating tf
        words = sentence.split()
        for ch in words:
            if ch in self.col:
                if ch in word_count:
                    word_count[ch] += 1
                else:
                    word_count[ch] = 1
    
        rough_tfidf=list(self.col)
        for keys in word_count.keys():
            tf_idf[keys]=idf[keys]*word_count[keys]
            if keys in rough_tfidf:
                index=rough_tfidf.index(keys)
                rough_tfidf[index]=tf_idf[keys]
        norm=0
        for i in range(0,len(rough_tfidf)):
            norm+=rough_tfidf[i]**2
        if norm==0:
            norm=1
        for i in range(0,len(rough_tfidf)):
            rough_tfidf[i]=round(rough_tfidf[i]/math.sqrt(norm),8)
        return rough_tfidf
    
    def count_sentence_eng(self):
        cnt=self.paragraph.count(".")+1
        return cnt
    def count_sentence_nep(self):
        cnt=self.paragraph.count("।")+1
        return cnt
    def count_summarized(self,summarized):
        cnt=summarized.count("||")+1
        return cnt
    
    def sentence_number(self,number):
        paragraph=self.paragraph
        
        if self.language=="en":
            processed_paragraph=self.english_process_paragraph()
            each_sentence=paragraph.split(".")
            each_sentence.remove("")
            sentence_count=self.count_sentence_eng()
        else:
            processed_paragraph=self.nepali_process_paragraph()
            #paragraph=paragraph.replace("।","|")
            each_sentence=paragraph.split("।")
            each_sentence.remove("")
            sentence_count=self.count_sentence_nep()
            
        
        if (number>sentence_count):
            print("ERROR: Summarization line exceeds total sentence count")
        
        else:
            #Each sentence tf_idf
            each_tfidf=[]
            tf_idf_score=[]
            for i in range(len(each_sentence)):
                each_tfidf.append(self.calc_tf_idf(each_sentence[i]))
            for i in range(len(each_tfidf)):
                each_tfidf[i]=np.array(each_tfidf[i])
                tf_idf_score.append(each_tfidf[i].sum())
            
            tf_idf_score=sorted(tf_idf_score)
            print(tf_idf_score)
            selection=tf_idf_score[-number:]
            
            summarized=""
            for i in range(sentence_count-1):
                for j in range(len(selection)-1,-1,-1):
                    if selection[j]==tf_idf_score[i]:
                        summarized += each_sentence[i]
                        summarized += "||"
                        break
                if self.count_summarized(summarized)==number+1:
                    break
                           
            return summarized 
        

In [133]:
class SummarizeCheck:
    def __init__(self,paragraph):
        self.vec=TfidfVectorizer()
        self.vec.fit_transform([paragraph])
        self.col=self.vec.get_feature_names_out()
        self.paragraph=paragraph
        self.language=detect(self.paragraph)
        if detect(self.language) != "en":
            #stop_words
            self.stop_words=open("stopwords.txt","r",encoding="utf-8")
            self.stop_words=self.stop_words.read()
            self.stop_words=self.stop_words.split("\n")
            
            #num file
            self.nepali_num=open("numbers.txt","r",encoding="utf-8")
            self.nepali_num=self.nepali_num.read()
            self.nepali_num=self.nepali_num.split(",")
            
            #suffix file
            self.nepali_suffix=open("suffix.txt","r",encoding="utf-8")
            self.nepali_suffix=self.nepali_suffix.read()
            self.nepali_suffix=self.nepali_suffix.split("\n")
    
    def nepali_process_paragraph(self):
        paragraph=str(self.paragraph)
        
        #removing \n and \ufeff
        remove=['\n','\ufeff']
        for i in remove:
            paragraph.replace(i,'')
        
        #read stop words
        #Remove Stop Words
        #word_tokens = Tokenizer().word_tokenize(text)
        word_tokens = paragraph.split(" ")
        filtered_list = [w for w in word_tokens if not w in self.stop_words]
        
        #Remove Nepali numbers
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(self.nepali_num)):
                if self.nepali_num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        for filter in num_filter:
            filtered_list.remove(filter)
        
        #Remove English numbers
        num=['0','1','2','3','4','5','6','7','8','9']
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(num)):
                if num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        for filter in num_filter:
            filtered_list.remove(filter)
        stemmed_string=' '.join(filtered_list)
        
        return stemmed_string
            
    def english_process_paragraph(self):
        paragraph=str(self.paragraph)
        #lowercasing
        paragraph=paragraph.lower()
        #Remove Stop Words
        stop_words=set(stopwords.words('english'))
        word_tokens = word_tokenize(paragraph)
        filtered_list = [w for w in word_tokens if not w in stop_words]
        
        
        #Remove numbers and special Symbols
        #words like 100m 2m were not removed so using this
        num=['0','1','2','3','4','5','6','7','8','9']
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(num)):
                if num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        
        for filter in num_filter:
            filtered_list.remove(filter)
                    
        filtered_list = [w for w in filtered_list if w.isalnum()]
        
        #Lematizing
        wordnet_lemmatizer=WordNetLemmatizer()
        lemmatized_list=[wordnet_lemmatizer.lemmatize(w,wordnet.VERB) for w in filtered_list]
        lemmatized_string=' '.join(lemmatized_list)
        
        return lemmatized_string
    
    def calc_idf(self):
        doc_count=len(self.paragraph)
        df={}
        idf={}
        for char in self.col:
            df[char]=0
            idf[char]=0
        #Calculating df
        for i in range(0,len(self.col)):
            for j in range(0,len(self.paragraph)):
                if self.col[i] in self.paragraph[j]:
                    df[self.col[i]]+=1
        #Calculating idf
        for char in self.col:
            idf[char]=math.log((doc_count+1)/(1+df[char]))+1
        #print(idf)
        #print(len(idf))
        #print("\n \n \n \n")
        #print(idf)
        return(idf)
    
    def calc_tf_idf(self,sentence):
        idf=self.calc_idf()
        
        tf_idf={}
        word_count={}
        '''
        for ch in self.col:
            tf_idf[ch]=0
            word_count[ch]=0
        #Calculating tf
        words = sentence.split()
        for ch in words:
            if ch in self.col:
                if ch in word_count:
                    word_count[ch] += 1
                else:
                    word_count[ch] = 1'''
        for i in range(len(sentence)):
            for t in sentence[i].split():
                if t not in word_count.keys():
                    word_count[t] = 1
                else:
                    word_count[t] +=1
    
        rough_tfidf=list(self.col)
        for keys in word_count.keys():
            tf_idf[keys]=idf[keys]*word_count[keys]
            if keys in rough_tfidf:
                index=rough_tfidf.index(keys)
                rough_tfidf[index]=tf_idf[keys]
        norm=0
        for i in range(0,len(rough_tfidf)):
            norm+=rough_tfidf[i]**2
        if norm==0:
            norm=1
        for i in range(0,len(rough_tfidf)):
            rough_tfidf[i]=round(rough_tfidf[i]/math.sqrt(norm),8)
        return rough_tfidf
    
    def count_sentence_eng(self):
        cnt=self.paragraph.count(".")+1
        return cnt
    def count_sentence_nep(self):
        cnt=self.paragraph.count("।")+1
        return cnt
    def count_summarized(self,summarized):
        cnt=summarized.count("||")+1
        return cnt
    
    def sentence_number(self,number):
        paragraph=self.paragraph
        
        if self.language=="en":
            processed_paragraph=self.english_process_paragraph()
            each_sentence=paragraph.split(".")
            each_sentence.remove("")
            sentence_count=self.count_sentence_eng()
        else:
            processed_paragraph=self.nepali_process_paragraph()
            paragraph=paragraph.replace("।","|")
            each_sentence=paragraph.split("|")
            each_sentence.remove("")
            sentence_count=self.count_sentence_nep()
            
        
        if (number>sentence_count):
            print("ERROR: Summarization line exceeds total sentence count")
        
        else:
            #Each sentence tf_idf
            each_tfidf=[]
            tf_idf_score=[]
            for i in range(len(each_sentence)):
                each_tfidf.append(self.calc_tf_idf(each_sentence[i]))
            for i in range(len(each_tfidf)):
                each_tfidf[i]=np.array(each_tfidf[i])
                tf_idf_score.append(each_tfidf[i].sum())
            
            tf_idf_score=sorted(tf_idf_score)
            print(tf_idf_score)
            selection=tf_idf_score[-number:]
            loopsteps = len(selection) - number
            j = len(selection) - 1 
            summarized=""
            for i in range(sentence_count-1):
                while j > loopsteps:
                    if selection[j]==tf_idf_score[i]:
                        summarized += each_sentence[i]
                        summarized += "||"
                        break
                    j -= 1           
            return summarized 
        

In [134]:
textnep="होवार्ड स्नूकर खेल्न आक्रोशित कन्जरभेटिभ नेता माइकल होवार्डले स्नूकर हलमा आफ्ना स्कूलका साथीहरूसँग समय बिताउन ट्रन्ट खेल्ने गरेको स्वीकार गरेका छन्। श्री होवार्डले भने कि 1950 को दशकमा लानेलीको ज्याक स्नूकर हलमा उनको समयले उनलाई कुनै दिगो नोक्सान पुर्याएन। तर उनले टाइम्स शैक्षिक सप्लिमेन्टलाई भने कि ट्र्यान्सी धेरै नराम्रो थियो र दृढ कार्य आवश्यक छ भने। मिस्टर होवर्डले ओ-लेभल र थप कक्षाकोठा अनुशासनमा फर्कन पनि आह्वान गरे। मिस्टर हावर्डले अन्ततः क्याम्ब्रिज विश्वविद्यालय जान लानेली व्याकरण स्कूल र स्नूकर हल छोडे। उसले भन्यो: मलाई लाग्दैन कि यसले मलाई कुनै दिगो नोक्सान गरेको छ। न त यसले मलाई स्नूकर विश्व च्याम्पियन बनाएको छ। हामी दिउँसो छिट्टै निस्किदा केही अवसरहरू भएका हुन सक्छन्। म केवल इमानदार छु। मलाई लाग्छ कि ट्र्यान्सी एक धेरै नराम्रो कुरा हो र यसको सामना गर्न दृढ कदम चाल्नु पर्छ। स्नूकरको विश्व च्याम्पियनसिप जित्न असफल भएका अर्को खेलाडी - जिमी द व्हर्लविन्ड ह्वाइट -ले पहिले आफ्नो दिन धुवाँदार हलहरूमा बिताउनको सट्टा पाठ हराएको स्वीकार गरेका छन्। टोनी मेयो [अर्को खेलाडी] र म हाम्रो सबै खाली समय त्यहाँ बिताउन गर्थ्यौं श्री ह्वाइटले भने कि हामी खेल र वातावरण मन पराउँछौं। विद्यालय झ्यालबाट बाहिर गयो । म केही समयको लागि गएँ र त्यसपछि समय लिन थाले। मिस्टर हावर्डको फेलो वेल्शम्यान रे रीर्डन - जसलाई उनका साथी पेशेवरहरू ड्र्याकुला भनेर चिनिन्छन् - स्नूकर विश्व च्याम्पियनशिप छ पटक जितेका थिए जसले १४ वर्षको उमेरमा स्कूल छोडेर खानीको रूपमा काम गरे। र ल्यानेलीका मिस्टर होवार्ड जस्ता टेरी ग्रिफिथले सन् १९७९ मा प्रतियोगिता जितेका थिए। यी दुईबीच ज्याक एसमा कहिले भिडन्त भएको थियो वा थिएन भन्ने थाहा छैन।"
texteng="howard  truanted to play snooker  conservative leader michael howard has admitted he used to play truant to spend time with his school friends at a snooker hall.  mr howard said his time at jack s snooker hall in llanelli in the 1950s had not done him  any lasting damage . but he told the times educational supplement that truancy was  very bad  and said  firm action  was needed. mr howard also called for a return to o-levels and more classroom discipline.  mr howard eventually left llanelli grammar school - and the snooker hall - to go to cambridge university. he said:  i don t think it s done me any lasting damage. nor has it made me a snooker world champion.  there might have been some occasions when we left early of an afternoon.   i m just being honest. i think truancy is a very bad thing and that firm action should be taken to deal with it.  another player who has failed to win snooker s world championship - jimmy  the whirlwind   white - has previously admitted missing lessons  instead spending his days in smoky halls.  tony meo [another player] and me used to spend all of our spare time there   mr white said   we loved the game and the atmosphere.  school went out of the window. i went for a while and then started taking time off.  mr howard s fellow welshman ray reardon - known by his fellow professionals as  dracula  - won the snooker world championship six times  having left school at 14 to work as a miner. and terry griffiths  like mr howard from llanelli  won the tournament in 1979. it is not known whether the two of them ever clashed cues at jack s."

In [135]:
texteng2="china had role in yukos split-up china lent russia $6bn (Â£3.2bn) to help the russian government renationalise the key yuganskneftegas unit of oil group yukos  it has been revealed.  the kremlin said on tuesday that the $6bn which russian state bank veb lent state-owned rosneft to help buy yugansk in turn came from chinese banks. the revelation came as the russian government said rosneft had signed a long-term oil supply deal with china. the deal sees rosneft receive $6bn in credits from china s cnpc.  according to russian newspaper vedomosti  these credits would be used to pay off the loans rosneft received to finance the purchase of yugansk. reports said cnpc had been offered 20% of yugansk in return for providing finance but the company opted for a long-term oil supply deal instead. analysts said one factor that might have influenced the chinese decision was the possibility of litigation from yukos  yugansk s former owner  if cnpc had become a shareholder. rosneft and veb declined to comment.  the two companies [rosneft and cnpc] have agreed on the pre-payment for long-term deliveries   said russian oil official sergei oganesyan.  there is nothing unusual that the pre-payment is for five to six years.   the announcements help to explain how rosneft  a medium-sized  indebted  and relatively unknown firm  was able to finance its surprise purchase of yugansk. yugansk was sold for $9.3bn in an auction last year to help yukos pay off part of a $27bn bill in unpaid taxes and fines.  the embattled russian oil giant had previously filed for bankruptcy protection in a us court in an attempt to prevent the forced sale of its main production arm. but yugansk was sold to a little known shell company which in turn was bought by rosneft. yukos claims its downfall was punishment for the political ambitions of its founder mikhail khodorkovsky. once the country s richest man  mr khodorkovsky is on trial for fraud and tax evasion.  the deal between rosneft and cnpc is seen as part of china s desire to secure long-term oil supplies to feed its booming economy. china s thirst for products such as crude oil  copper and steel has helped pushed global commodity prices to record levels.  clearly the chinese are trying to get some leverage [in russia]   said dmitry lukashov  an analyst at brokerage aton.  they understand property rights in russia are not the most important rights  and they are more interested in guaranteeing supplies.   if the price of oil is fixed under the deal  which is unlikely  it could be very profitable for the chinese   mr lukashov continued.  and rosneft is in desperate need of cash  so it s a good deal for them too."
textnep2="युकोस विभाजनमा चीनको भूमिका रहेको थियो, चीनले रुसी सरकारलाई तेल समूह युकोसको प्रमुख युगान्स्कनेफ्तेगास एकाइलाई पुनः राष्ट्रियकरण गर्न मद्दत गर्न रुसलाई ६ अर्ब डलर (£३.२ बिलियन) ऋण दिएको खुलासा भएको छ । क्रेमलिनले मंगलबार भन्यो कि रुसी स्टेट बैंक भेबले युगान्स्क किन्न मद्दत गर्न सरकारी स्वामित्वको रोजनेफ्टलाई 6 बिलियन डलर चिनियाँ बैंकहरूबाट आएको हो। रुसी सरकारले रोस्नेफ्टले चीनसँग दीर्घकालीन तेल आपूर्ति सम्झौतामा हस्ताक्षर गरेको भन्दै यो खुलासा भएको हो। सम्झौताले रोस्नेफ्टले चीनको सीएनपीसीबाट $ 6 बिलियन क्रेडिट प्राप्त गरेको देख्छ। रुसी अखबार वेदोमोस्तीका अनुसार यी क्रेडिटहरू रोसनेफ्टले युगान्स्कको खरिदका लागि प्राप्त गरेको ऋण तिर्न प्रयोग गरिने छ। रिपोर्टहरूका अनुसार सीएनपीसीलाई वित्त प्रदान गर्न बदलामा युगान्स्कको २०% प्रस्ताव गरिएको थियो तर कम्पनीले यसको सट्टा दीर्घकालीन तेल आपूर्ति सम्झौताको लागि रोजेको थियो। विश्लेषकहरूले चिनियाँ निर्णयलाई प्रभाव पार्ने एउटा कारक युकोस युगान्स्कको पूर्व मालिक सीएनपीसी सेयरहोल्डर भएको खण्डमा मुद्दा चलाउने सम्भावना रहेको बताए। rosneft र veb टिप्पणी गर्न अस्वीकार गरे। दुई कम्पनीहरू [rosneft र cnpc] लामो अवधिको डेलिभरीको लागि पूर्व भुक्तानीमा सहमत भएको रूसी तेल अधिकारी सर्गेई ओगानेसियानले बताए। पूर्व-भुक्तानी पाँच देखि छ वर्षको लागि हो भन्ने असामान्य केही छैन। घोषणाहरूले कसरी मध्यम आकारको ऋणी र अपेक्षाकृत अज्ञात फर्मले युगान्स्कको आफ्नो अचम्मको खरीदलाई वित्त पोषण गर्न सक्षम भयो भनेर व्याख्या गर्न मद्दत गर्दछ। गत वर्ष एक लिलामीमा yugansk $ 9.3 बिलियनमा बेचेको थियो जसले युकोहरूलाई नतिरेको कर र जरिवानामा $ 27 बिलियन बिलको अंश तिर्न मद्दत गर्दछ। द्वन्द्वग्रस्त रूसी तेल दिग्गजले पहिले आफ्नो मुख्य उत्पादन हातको जबरजस्ती बिक्री रोक्न प्रयासमा अमेरिकी अदालतमा दिवालियापन सुरक्षाको लागि दायर गरेको थियो। तर युगान्स्क थोरै ज्ञात शेल कम्पनीलाई बेचेको थियो जुन फलस्वरूप रोजनेफ्टले किनेको थियो। युकोसले यसको पतन यसको संस्थापक मिखाइल खोडोरकोभस्कीको राजनीतिक महत्वाकांक्षाको लागि सजाय भएको दाबी गर्दछ। एक पटक देशको सबैभन्दा धनी व्यक्ति मिस्टर खोडोरकोभस्की ठगी र कर छलीको लागि मुद्दामा छन्। रोस्नेफ्ट र सीएनपीसी बीचको सम्झौतालाई चीनको बढ्दो अर्थतन्त्रलाई खुवाउन दीर्घकालीन तेल आपूर्ति सुरक्षित गर्ने इच्छाको अंशको रूपमा हेरिएको छ। कच्चा तेल तामा र स्टिल जस्ता उत्पादनहरूका लागि चीनको तिर्खाले विश्वव्यापी वस्तुको मूल्यलाई रेकर्ड स्तरमा धकेल्न मद्दत गरेको छ। ब्रोकरेज एटोनका विश्लेषक दिमित्री लुकाशोभले स्पष्ट रूपमा चिनियाँहरूले [रूसमा] केही फाइदा लिन खोजिरहेका छन्। तिनीहरू बुझ्छन् रसियामा सम्पत्ति अधिकारहरू सबैभन्दा महत्त्वपूर्ण अधिकारहरू होइनन् र तिनीहरू आपूर्तिको ग्यारेन्टीमा बढी चासो राख्छन्। यदि यो सम्झौता अन्तर्गत तेलको मूल्य निर्धारण गरिएको छ जुन असम्भव छ यो चिनियाँ श्री लुकाशोभको लागि धेरै लाभदायक हुन सक्छ। र रोस्नेफ्टलाई नगदको असाध्यै आवश्यकता छ त्यसैले यो उनीहरूको लागि पनि राम्रो सम्झौता हो।"

In [136]:
print(Summarize(textnep).count_sentence_nep())
Summarize(texteng).count_sentence_eng()

18


18

In [137]:
print(Summarize(textnep).sentence_number(1))
Summarize(texteng).sentence_number(1)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.41421356, 1.41421356, 1.41421356]
 मिस्टर हावर्डको फेलो वेल्शम्यान रे रीर्डन - जसलाई उनका साथी पेशेवरहरू ड्र्याकुला भनेर चिनिन्छन् - स्नूकर विश्व च्याम्पियनशिप छ पटक जितेका थिए जसले १४ वर्षको उमेरमा स्कूल छोडेर खानीको रूपमा काम गरे||
[1.73205081, 2.44948974, 2.82842712, 2.9999999699999993, 2.99999997, 3.31662474, 3.6055512999999997, 3.6055512999999997, 3.74165736, 3.77123616, 4.0249224, 4.12310571, 4.21775692, 4.37880262, 4.42635207, 4.91353823, 5.04825212]


' it is not known whether the two of them ever clashed cues at jack s||'

In [138]:
textnep="होवार्ड स्नूकर खेल्न आक्रोशित कन्जरभेटिभ नेता माइकल होवार्डले स्नूकर हलमा आफ्ना स्कूलका साथीहरूसँग समय बिताउन ट्रन्ट खेल्ने गरेको स्वीकार गरेका छन्। श्री होवार्डले भने कि 1950 को दशकमा लानेलीको ज्याक स्नूकर हलमा उनको समयले उनलाई कुनै दिगो नोक्सान पुर्याएन। तर उनले टाइम्स शैक्षिक सप्लिमेन्टलाई भने कि ट्र्यान्सी धेरै नराम्रो थियो र दृढ कार्य आवश्यक छ भने। मिस्टर होवर्डले ओ-लेभल र थप कक्षाकोठा अनुशासनमा फर्कन पनि आह्वान गरे। मिस्टर हावर्डले अन्ततः क्याम्ब्रिज विश्वविद्यालय जान लानेली व्याकरण स्कूल र स्नूकर हल छोडे। उसले भन्यो: मलाई लाग्दैन कि यसले मलाई कुनै दिगो नोक्सान गरेको छ। न त यसले मलाई स्नूकर विश्व च्याम्पियन बनाएको छ। हामी दिउँसो छिट्टै निस्किदा केही अवसरहरू भएका हुन सक्छन्। म केवल इमानदार छु। मलाई लाग्छ कि ट्र्यान्सी एक धेरै नराम्रो कुरा हो र यसको सामना गर्न दृढ कदम चाल्नु पर्छ। स्नूकरको विश्व च्याम्पियनसिप जित्न असफल भएका अर्को खेलाडी - जिमी द व्हर्लविन्ड ह्वाइट -ले पहिले आफ्नो दिन धुवाँदार हलहरूमा बिताउनको सट्टा पाठ हराएको स्वीकार गरेका छन्। टोनी मेयो [अर्को खेलाडी] र म हाम्रो सबै खाली समय त्यहाँ बिताउन गर्थ्यौं श्री ह्वाइटले भने कि हामी खेल र वातावरण मन पराउँछौं। विद्यालय झ्यालबाट बाहिर गयो । म केही समयको लागि गएँ र त्यसपछि समय लिन थाले। मिस्टर हावर्डको फेलो वेल्शम्यान रे रीर्डन - जसलाई उनका साथी पेशेवरहरू ड्र्याकुला भनेर चिनिन्छन् - स्नूकर विश्व च्याम्पियनशिप छ पटक जितेका थिए जसले १४ वर्षको उमेरमा स्कूल छोडेर खानीको रूपमा काम गरे। र ल्यानेलीका मिस्टर होवार्ड जस्ता टेरी ग्रिफिथले सन् १९७९ मा प्रतियोगिता जितेका थिए। यी दुईबीच ज्याक एसमा कहिले भिडन्त भएको थियो वा थिएन भन्ने थाहा छैन।"
print(textnep)

होवार्ड स्नूकर खेल्न आक्रोशित कन्जरभेटिभ नेता माइकल होवार्डले स्नूकर हलमा आफ्ना स्कूलका साथीहरूसँग समय बिताउन ट्रन्ट खेल्ने गरेको स्वीकार गरेका छन्। श्री होवार्डले भने कि 1950 को दशकमा लानेलीको ज्याक स्नूकर हलमा उनको समयले उनलाई कुनै दिगो नोक्सान पुर्याएन। तर उनले टाइम्स शैक्षिक सप्लिमेन्टलाई भने कि ट्र्यान्सी धेरै नराम्रो थियो र दृढ कार्य आवश्यक छ भने। मिस्टर होवर्डले ओ-लेभल र थप कक्षाकोठा अनुशासनमा फर्कन पनि आह्वान गरे। मिस्टर हावर्डले अन्ततः क्याम्ब्रिज विश्वविद्यालय जान लानेली व्याकरण स्कूल र स्नूकर हल छोडे। उसले भन्यो: मलाई लाग्दैन कि यसले मलाई कुनै दिगो नोक्सान गरेको छ। न त यसले मलाई स्नूकर विश्व च्याम्पियन बनाएको छ। हामी दिउँसो छिट्टै निस्किदा केही अवसरहरू भएका हुन सक्छन्। म केवल इमानदार छु। मलाई लाग्छ कि ट्र्यान्सी एक धेरै नराम्रो कुरा हो र यसको सामना गर्न दृढ कदम चाल्नु पर्छ। स्नूकरको विश्व च्याम्पियनसिप जित्न असफल भएका अर्को खेलाडी - जिमी द व्हर्लविन्ड ह्वाइट -ले पहिले आफ्नो दिन धुवाँदार हलहरूमा बिताउनको सट्टा पाठ हराएको स्वीकार गरेका छन्। टोनी मेयो [अर्को खेलाडी] र म हाम्रो 

In [139]:
print(SummarizeCheck(textnep).sentence_number(1))
SummarizeCheck(texteng).sentence_number(1)

KeyError: 'ह'

In [ ]:
print(Summarize(texteng2).count_sentence_eng())
Summarize(textnep2).count_sentence_nep()

25


23

In [ ]:
print(Summarize(texteng2).sentence_number(4))
Summarize(textnep2).sentence_number(4)

[2.0, 2.44948974, 2.52982215, 3.1622777, 3.20713488, 3.63803444, 3.7712361599999995, 3.7712361599999995, 3.7712361599999995, 3.77964476, 3.87298335, 3.87298335, 4.0249223999999995, 4.12310571, 4.13092191, 4.14613991, 4.24264078, 4.37880262, 4.472136, 4.49073129, 4.49073129, 4.70678735, 4.8493431000000005, 4.91353823]
 china s thirst for products such as crude oil  copper and steel has helped pushed global commodity prices to record levels||  clearly the chinese are trying to get some leverage [in russia]   said dmitry lukashov  an analyst at brokerage aton||  they understand property rights in russia are not the most important rights  and they are more interested in guaranteeing supplies||   if the price of oil is fixed under the deal  which is unlikely  it could be very profitable for the chinese   mr lukashov continued||
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.41421356, 1.41421356, 1.73205081, 2.236068]


' ब्रोकरेज एटोनका विश्लेषक दिमित्री लुकाशोभले स्पष्ट रूपमा चिनियाँहरूले [रूसमा] केही फाइदा लिन खोजिरहेका छन्|| तिनीहरू बुझ्छन् रसियामा सम्पत्ति अधिकारहरू सबैभन्दा महत्त्वपूर्ण अधिकारहरू होइनन् र तिनीहरू आपूर्तिको ग्यारेन्टीमा बढी चासो राख्छन्|| यदि यो सम्झौता अन्तर्गत तेलको मूल्य निर्धारण गरिएको छ जुन असम्भव छ यो चिनियाँ श्री लुकाशोभको लागि धेरै लाभदायक हुन सक्छ|| र रोस्नेफ्टलाई नगदको असाध्यै आवश्यकता छ त्यसैले यो उनीहरूको लागि पनि राम्रो सम्झौता हो||'

In [ ]:
def nepalipree(nep):
    paragraph=nep
    paragraph=paragraph.replace("।","|")
    each_sentence=paragraph.split("|")
    each_sentence.remove("")
    each_tfidf=[]
    print(each_sentence)
    tf_idf_score=[]
    for i in range(len(each_sentence)):
        each_tfidf.append(Summarize(nep).calc_tf_idf(each_sentence[i]))
    for i in range(len(each_tfidf)):
        each_tfidf[i]=np.array(each_tfidf[i])
        tf_idf_score.append(each_tfidf[i].sum())
    print(tf_idf_score)
    tf_idf_score=sorted(tf_idf_score)
    
    selection=tf_idf_score[-4:]
    
    summarized=""
    for i in range(18-1):
                for j in range(len(selection)):
                    if selection[j]==tf_idf_score[i]:
                        summarized += each_sentence[i]
                        summarized += "||"
                        break            
    return summarized 
    

In [ ]:
textnep="होवार्ड स्नूकर खेल्न आक्रोशित कन्जरभेटिभ नेता माइकल होवार्डले स्नूकर हलमा आफ्ना स्कूलका साथीहरूसँग समय बिताउन ट्रन्ट खेल्ने गरेको स्वीकार गरेका छन्। श्री होवार्डले भने कि 1950 को दशकमा लानेलीको ज्याक स्नूकर हलमा उनको समयले उनलाई कुनै दिगो नोक्सान पुर्याएन। तर उनले टाइम्स शैक्षिक सप्लिमेन्टलाई भने कि ट्र्यान्सी धेरै नराम्रो थियो र दृढ कार्य आवश्यक छ भने। मिस्टर होवर्डले ओ-लेभल र थप कक्षाकोठा अनुशासनमा फर्कन पनि आह्वान गरे। मिस्टर हावर्डले अन्ततः क्याम्ब्रिज विश्वविद्यालय जान लानेली व्याकरण स्कूल र स्नूकर हल छोडे। उसले भन्यो: मलाई लाग्दैन कि यसले मलाई कुनै दिगो नोक्सान गरेको छ। न त यसले मलाई स्नूकर विश्व च्याम्पियन बनाएको छ। हामी दिउँसो छिट्टै निस्किदा केही अवसरहरू भएका हुन सक्छन्। म केवल इमानदार छु। मलाई लाग्छ कि ट्र्यान्सी एक धेरै नराम्रो कुरा हो र यसको सामना गर्न दृढ कदम चाल्नु पर्छ। स्नूकरको विश्व च्याम्पियनसिप जित्न असफल भएका अर्को खेलाडी - जिमी द व्हर्लविन्ड ह्वाइट -ले पहिले आफ्नो दिन धुवाँदार हलहरूमा बिताउनको सट्टा पाठ हराएको स्वीकार गरेका छन्। टोनी मेयो [अर्को खेलाडी] र म हाम्रो सबै खाली समय त्यहाँ बिताउन गर्थ्यौं श्री ह्वाइटले भने कि हामी खेल र वातावरण मन पराउँछौं। विद्यालय झ्यालबाट बाहिर गयो । म केही समयको लागि गएँ र त्यसपछि समय लिन थाले। मिस्टर हावर्डको फेलो वेल्शम्यान रे रीर्डन - जसलाई उनका साथी पेशेवरहरू ड्र्याकुला भनेर चिनिन्छन् - स्नूकर विश्व च्याम्पियनशिप छ पटक जितेका थिए जसले १४ वर्षको उमेरमा स्कूल छोडेर खानीको रूपमा काम गरे। र ल्यानेलीका मिस्टर होवार्ड जस्ता टेरी ग्रिफिथले सन् १९७९ मा प्रतियोगिता जितेका थिए। यी दुईबीच ज्याक एसमा कहिले भिडन्त भएको थियो वा थिएन भन्ने थाहा छैन।"

In [ ]:
nepalipree(textnep)

['होवार्ड स्नूकर खेल्न आक्रोशित कन्जरभेटिभ नेता माइकल होवार्डले स्नूकर हलमा आफ्ना स्कूलका साथीहरूसँग समय बिताउन ट्रन्ट खेल्ने गरेको स्वीकार गरेका छन्', ' श्री होवार्डले भने कि 1950 को दशकमा लानेलीको ज्याक स्नूकर हलमा उनको समयले उनलाई कुनै दिगो नोक्सान पुर्याएन', ' तर उनले टाइम्स शैक्षिक सप्लिमेन्टलाई भने कि ट्र्यान्सी धेरै नराम्रो थियो र दृढ कार्य आवश्यक छ भने', ' मिस्टर होवर्डले ओ-लेभल र थप कक्षाकोठा अनुशासनमा फर्कन पनि आह्वान गरे', ' मिस्टर हावर्डले अन्ततः क्याम्ब्रिज विश्वविद्यालय जान लानेली व्याकरण स्कूल र स्नूकर हल छोडे', ' उसले भन्यो: मलाई लाग्दैन कि यसले मलाई कुनै दिगो नोक्सान गरेको छ', ' न त यसले मलाई स्नूकर विश्व च्याम्पियन बनाएको छ', ' हामी दिउँसो छिट्टै निस्किदा केही अवसरहरू भएका हुन सक्छन्', ' म केवल इमानदार छु', ' मलाई लाग्छ कि ट्र्यान्सी एक धेरै नराम्रो कुरा हो र यसको सामना गर्न दृढ कदम चाल्नु पर्छ', ' स्नूकरको विश्व च्याम्पियनसिप जित्न असफल भएका अर्को खेलाडी - जिमी द व्हर्लविन्ड ह्वाइट -ले पहिले आफ्नो दिन धुवाँदार हलहरूमा बिताउनको सट्टा पाठ हराएको स्वीकार गरेका छन्', ' ट

' न त यसले मलाई स्नूकर विश्व च्याम्पियन बनाएको छ|| हामी दिउँसो छिट्टै निस्किदा केही अवसरहरू भएका हुन सक्छन्|| म केवल इमानदार छु|| मलाई लाग्छ कि ट्र्यान्सी एक धेरै नराम्रो कुरा हो र यसको सामना गर्न दृढ कदम चाल्नु पर्छ|| स्नूकरको विश्व च्याम्पियनसिप जित्न असफल भएका अर्को खेलाडी - जिमी द व्हर्लविन्ड ह्वाइट -ले पहिले आफ्नो दिन धुवाँदार हलहरूमा बिताउनको सट्टा पाठ हराएको स्वीकार गरेका छन्|| टोनी मेयो [अर्को खेलाडी] र म हाम्रो सबै खाली समय त्यहाँ बिताउन गर्थ्यौं श्री ह्वाइटले भने कि हामी खेल र वातावरण मन पराउँछौं|| विद्यालय झ्यालबाट बाहिर गयो || म केही समयको लागि गएँ र त्यसपछि समय लिन थाले|| मिस्टर हावर्डको फेलो वेल्शम्यान रे रीर्डन - जसलाई उनका साथी पेशेवरहरू ड्र्याकुला भनेर चिनिन्छन् - स्नूकर विश्व च्याम्पियनशिप छ पटक जितेका थिए जसले १४ वर्षको उमेरमा स्कूल छोडेर खानीको रूपमा काम गरे|| र ल्यानेलीका मिस्टर होवार्ड जस्ता टेरी ग्रिफिथले सन् १९७९ मा प्रतियोगिता जितेका थिए|| यी दुईबीच ज्याक एसमा कहिले भिडन्त भएको थियो वा थिएन भन्ने थाहा छैन||'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vect = TfidfVectorizer()

In [ ]:
x=vect.fit_transform([textnep])

In [ ]:
y=x.toarray()

In [ ]:
y

array([[0.04962917, 0.09925833, 0.09925833, 0.04962917, 0.04962917,
        0.04962917, 0.09925833, 0.04962917, 0.04962917, 0.04962917,
        0.04962917, 0.04962917, 0.09925833, 0.04962917, 0.09925833,
        0.1488875 , 0.09925833, 0.04962917, 0.04962917, 0.1488875 ,
        0.09925833, 0.04962917, 0.04962917, 0.04962917, 0.09925833,
        0.297775  , 0.04962917, 0.04962917, 0.04962917, 0.04962917,
        0.04962917, 0.39703333, 0.19851667, 0.04962917, 0.04962917,
        0.09925833, 0.19851667, 0.04962917, 0.04962917, 0.09925833,
        0.19851667, 0.04962917, 0.04962917, 0.04962917, 0.04962917,
        0.04962917, 0.04962917, 0.09925833, 0.04962917, 0.04962917,
        0.04962917, 0.09925833, 0.04962917, 0.04962917, 0.04962917,
        0.1488875 , 0.34740417, 0.04962917, 0.09925833, 0.19851667,
        0.04962917, 0.04962917, 0.04962917, 0.04962917, 0.04962917,
        0.04962917, 0.09925833, 0.04962917, 0.04962917, 0.04962917,
        0.04962917, 0.04962917, 0.04962917, 0.09

In [ ]:
corpus=['होवार्ड स्नूकर खेल्न आक्रोशित कन्जरभेटिभ नेता माइकल होवार्डले स्नूकर हलमा आफ्ना स्कूलका साथीहरूसँग समय बिताउन ट्रन्ट खेल्ने गरेको स्वीकार गरेका छन्', ' श्री होवार्डले भने कि 1950 को दशकमा लानेलीको ज्याक स्नूकर हलमा उनको समयले उनलाई कुनै दिगो नोक्सान पुर्याएन', ' तर उनले टाइम्स शैक्षिक सप्लिमेन्टलाई भने कि ट्र्यान्सी धेरै नराम्रो थियो र दृढ कार्य आवश्यक छ भने']

In [ ]:
sum = Summarize(corpus[0])

In [ ]:
sum.calc_tf_idf(corpus[0])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]

In [ ]:
len(corpus[0].split())

21

In [ ]:
tv = {}
for t in corpus[0].split():
    if t not in tv.keys():
        tv[t] = 1
    else:
        tv[t] +=1

In [ ]:
tv

{'होवार्ड': 1,
 'स्नूकर': 2,
 'खेल्न': 1,
 'आक्रोशित': 1,
 'कन्जरभेटिभ': 1,
 'नेता': 1,
 'माइकल': 1,
 'होवार्डले': 1,
 'हलमा': 1,
 'आफ्ना': 1,
 'स्कूलका': 1,
 'साथीहरूसँग': 1,
 'समय': 1,
 'बिताउन': 1,
 'ट्रन्ट': 1,
 'खेल्ने': 1,
 'गरेको': 1,
 'स्वीकार': 1,
 'गरेका': 1,
 'छन्': 1}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

X = tfidf_vectorizer.fit_transform(corpus)
tfidf_vectorizer.get_feature_names_out()


array(['1950', 'आक', 'आफ', 'आवश', 'इकल', 'इम', 'उन', 'उनक', 'उनल', 'एन',
       'कन', 'कर', 'गर', 'छन', 'जरभ', 'टल', 'डल', 'तर', 'दशकम', 'नर',
       'भन', 'यक', 'रन', 'लक', 'सप', 'समय', 'समयल', 'हर', 'हलम'],
      dtype=object)

In [ ]:
X.shape

(3, 29)

In [ ]:
corpus = ['howard  truanted to play snooker  conservative leader michael howard has admitted he used to play truant to spend time with his school friends at a snooker hall.','mr howard said his time at jack s  snooker hall in llanelli in the 1950s had not done him  any lasting damage.','but he told the times educational supplement that truancy was  very bad  and said  firm action  was needed.']

In [ ]:
X = tfidf_vectorizer.fit_transform(corpus)
tfidf_vectorizer.get_feature_names_out()

array(['1950s', 'action', 'admitted', 'and', 'any', 'at', 'bad', 'but',
       'conservative', 'damage', 'done', 'educational', 'firm', 'friends',
       'had', 'hall', 'has', 'he', 'him', 'his', 'howard', 'in', 'jack',
       'lasting', 'leader', 'llanelli', 'michael', 'mr', 'needed', 'not',
       'play', 'said', 'school', 'snooker', 'spend', 'supplement', 'that',
       'the', 'time', 'times', 'to', 'told', 'truancy', 'truant',
       'truanted', 'used', 'very', 'was', 'with'], dtype=object)

In [ ]:
print(X.shape)

(3, 49)


In [ ]:
if 'होवार्ड' == 'होवार्ड':
    print('Yes')

Yes


In [ ]:
print('होवार्ड')

होवार्ड


In [176]:
class SummarizeCheck:
    def __init__(self,paragraph):
        self.paragraph=paragraph
        self.language=detect(self.paragraph)
        if detect(self.language) != "en":
            #stop_words
            self.stop_words=open("stopwords.txt","r",encoding="utf-8")
            self.stop_words=self.stop_words.read()
            self.stop_words=self.stop_words.split("\n")
            
            #num file
            self.nepali_num=open("numbers.txt","r",encoding="utf-8")
            self.nepali_num=self.nepali_num.read()
            self.nepali_num=self.nepali_num.split(",")
            
            #suffix file
            self.nepali_suffix=open("suffix.txt","r",encoding="utf-8")
            self.nepali_suffix=self.nepali_suffix.read()
            self.nepali_suffix=self.nepali_suffix.split("\n")
            
            self.paragraph = self.nepali_process_paragraph()
        else:
            self.paragraph = self.english_process_paragraph()
        self.col=[]
        for t in paragraph.split():
            if t not in self.col:
                self.col.append(t)
    
    def nepali_process_paragraph(self):
        paragraph=str(self.paragraph)
        
        #removing \n and \ufeff
        remove=['\n','\ufeff']
        for i in remove:
            paragraph.replace(i,'')
        
        #read stop words
        #Remove Stop Words
        #word_tokens = Tokenizer().word_tokenize(text)
        word_tokens = paragraph.split(" ")
        filtered_list = [w for w in word_tokens if not w in self.stop_words]
        
        #Remove Nepali numbers
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(self.nepali_num)):
                if self.nepali_num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        for filter in num_filter:
            filtered_list.remove(filter)
        
        #Remove English numbers
        num=['0','1','2','3','4','5','6','7','8','9']
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(num)):
                if num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        for filter in num_filter:
            filtered_list.remove(filter)
        stemmed_string=' '.join(filtered_list)
        
        return stemmed_string
            
    def english_process_paragraph(self):
        paragraph=str(self.paragraph)
        #lowercasing
        paragraph=paragraph.lower()
        #Remove Stop Words
        stop_words=set(stopwords.words('english'))
        word_tokens = word_tokenize(paragraph)
        filtered_list = [w for w in word_tokens if not w in stop_words]
        
        
        #Remove numbers and special Symbols
        #words like 100m 2m were not removed so using this
        num=['0','1','2','3','4','5','6','7','8','9']
        num_filter=[]
        for i in range(0,len(filtered_list)):
            for j in range(0,len(num)):
                if num[j] in filtered_list[i]:
                    num_filter.append(filtered_list[i])
                    break
        
        for filter in num_filter:
            filtered_list.remove(filter)
                    
        filtered_list = [w for w in filtered_list if w.isalnum()]
        
        #Lematizing
        wordnet_lemmatizer=WordNetLemmatizer()
        lemmatized_list=[wordnet_lemmatizer.lemmatize(w,wordnet.VERB) for w in filtered_list]
        lemmatized_string=' '.join(lemmatized_list)
        
        return lemmatized_string
    
    def calc_idf(self):
        doc_count=len(self.paragraph)
        df={}
        idf={}
        for char in self.col:
            df[char]=0
            idf[char]=0
        #Calculating df
        for i in range(0,len(self.col)):
            for j in range(0,len(self.paragraph)):
                if self.col[i] in self.paragraph[j]:
                    df[self.col[i]]+=1
        #Calculating idf
        for char in self.col:
            idf[char]=math.log((doc_count+1)/(1+df[char]))+1
        return(idf)
    
    def calc_tf_idf(self,sentence):
        idf=self.calc_idf()
        
        tf_idf={}
        word_count={}
    
        for ch in self.col:
            tf_idf[ch]=0
            word_count[ch]=0
        #Calculating tf
        words = sentence.split()
        for ch in words:
            if ch in self.col:
                if ch in word_count:
                    word_count[ch] += 1
                else:
                    word_count[ch] = 1
    
        rough_tfidf=list(self.col)
        for keys in word_count.keys():
            tf_idf[keys]=idf[keys]*word_count[keys]
            if keys in rough_tfidf:
                index=rough_tfidf.index(keys)
                rough_tfidf[index]=tf_idf[keys]
        norm=0
        for i in range(0,len(rough_tfidf)):
            norm+=rough_tfidf[i]**2
        if norm==0:
            norm=1
        for i in range(0,len(rough_tfidf)):
            rough_tfidf[i]=round(rough_tfidf[i]/math.sqrt(norm),8)
        return rough_tfidf
    
    def count_sentence_eng(self):
        cnt=self.paragraph.count(".")+1
        return cnt
    def count_sentence_nep(self):
        cnt=self.paragraph.count("।")+1
        return cnt
    def count_summarized(self,summarized):
        cnt=summarized.count("||")+1
        return cnt
    
    def sentence_number(self,number):
        paragraph=self.paragraph
        
        if self.language=="en":
            processed_paragraph=self.english_process_paragraph()
            each_sentence=paragraph.split(".")
            each_sentence.remove("")
            sentence_count=self.count_sentence_eng()
        else:
            processed_paragraph=self.nepali_process_paragraph()
            #paragraph=paragraph.replace("।","|")
            each_sentence=paragraph.split("।")
            each_sentence.remove("")
            sentence_count=self.count_sentence_nep()
            
        
        if (number>sentence_count):
            print("ERROR: Summarization line exceeds total sentence count")
        
        else:
            #Each sentence tf_idf
            each_tfidf=[]
            tf_idf_score=[]
            for i in range(len(each_sentence)):
                each_tfidf.append(self.calc_tf_idf(each_sentence[i]))
            for i in range(len(each_tfidf)):
                each_tfidf[i]=np.array(each_tfidf[i])
                tf_idf_score.append(each_tfidf[i].sum())
            
            tf_idf_score=sorted(tf_idf_score)
            print(tf_idf_score)
            selection=tf_idf_score[-number:]
            
            summarized=""
            for i in range(sentence_count-1):
                for j in range(len(selection)-1,-1,-1):
                    if selection[j]==tf_idf_score[i]:
                        summarized += each_sentence[i]
                        summarized += "||"
                        break
                if self.count_summarized(summarized)==number+1:
                    break
                           
            return summarized 
        

In [177]:
textnep="होवार्ड स्नूकर खेल्न आक्रोशित कन्जरभेटिभ नेता माइकल होवार्डले स्नूकर हलमा आफ्ना स्कूलका साथीहरूसँग समय बिताउन ट्रन्ट खेल्ने गरेको स्वीकार गरेका छन्। श्री होवार्डले भने कि 1950 को दशकमा लानेलीको ज्याक स्नूकर हलमा उनको समयले उनलाई कुनै दिगो नोक्सान पुर्याएन। तर उनले टाइम्स शैक्षिक सप्लिमेन्टलाई भने कि ट्र्यान्सी धेरै नराम्रो थियो र दृढ कार्य आवश्यक छ भने। मिस्टर होवर्डले ओ-लेभल र थप कक्षाकोठा अनुशासनमा फर्कन पनि आह्वान गरे। मिस्टर हावर्डले अन्ततः क्याम्ब्रिज विश्वविद्यालय जान लानेली व्याकरण स्कूल र स्नूकर हल छोडे। उसले भन्यो: मलाई लाग्दैन कि यसले मलाई कुनै दिगो नोक्सान गरेको छ। न त यसले मलाई स्नूकर विश्व च्याम्पियन बनाएको छ। हामी दिउँसो छिट्टै निस्किदा केही अवसरहरू भएका हुन सक्छन्। म केवल इमानदार छु। मलाई लाग्छ कि ट्र्यान्सी एक धेरै नराम्रो कुरा हो र यसको सामना गर्न दृढ कदम चाल्नु पर्छ। स्नूकरको विश्व च्याम्पियनसिप जित्न असफल भएका अर्को खेलाडी - जिमी द व्हर्लविन्ड ह्वाइट -ले पहिले आफ्नो दिन धुवाँदार हलहरूमा बिताउनको सट्टा पाठ हराएको स्वीकार गरेका छन्। टोनी मेयो [अर्को खेलाडी] र म हाम्रो सबै खाली समय त्यहाँ बिताउन गर्थ्यौं श्री ह्वाइटले भने कि हामी खेल र वातावरण मन पराउँछौं। विद्यालय झ्यालबाट बाहिर गयो । म केही समयको लागि गएँ र त्यसपछि समय लिन थाले। मिस्टर हावर्डको फेलो वेल्शम्यान रे रीर्डन - जसलाई उनका साथी पेशेवरहरू ड्र्याकुला भनेर चिनिन्छन् - स्नूकर विश्व च्याम्पियनशिप छ पटक जितेका थिए जसले १४ वर्षको उमेरमा स्कूल छोडेर खानीको रूपमा काम गरे। र ल्यानेलीका मिस्टर होवार्ड जस्ता टेरी ग्रिफिथले सन् १९७९ मा प्रतियोगिता जितेका थिए। यी दुईबीच ज्याक एसमा कहिले भिडन्त भएको थियो वा थिएन भन्ने थाहा छैन।"
texteng="howard  truanted to play snooker  conservative leader michael howard has admitted he used to play truant to spend time with his school friends at a snooker hall.  mr howard said his time at jack s snooker hall in llanelli in the 1950s had not done him  any lasting damage . but he told the times educational supplement that truancy was  very bad  and said  firm action  was needed. mr howard also called for a return to o-levels and more classroom discipline.  mr howard eventually left llanelli grammar school - and the snooker hall - to go to cambridge university. he said:  i don t think it s done me any lasting damage. nor has it made me a snooker world champion.  there might have been some occasions when we left early of an afternoon.   i m just being honest. i think truancy is a very bad thing and that firm action should be taken to deal with it.  another player who has failed to win snooker s world championship - jimmy  the whirlwind   white - has previously admitted missing lessons  instead spending his days in smoky halls.  tony meo [another player] and me used to spend all of our spare time there   mr white said   we loved the game and the atmosphere.  school went out of the window. i went for a while and then started taking time off.  mr howard s fellow welshman ray reardon - known by his fellow professionals as  dracula  - won the snooker world championship six times  having left school at 14 to work as a miner. and terry griffiths  like mr howard from llanelli  won the tournament in 1979. it is not known whether the two of them ever clashed cues at jack s."

In [182]:
print(SummarizeCheck(texteng).sentence_number(5))
SummarizeCheck(textnep).sentence_number(5)

[1.89696827, 2.236068, 2.77242604, 3.06972982, 3.4180914099999997, 3.46410156, 3.49103474, 3.76052862, 3.90006742, 4.27740487, 4.2904784099999995, 4.34282796, 4.53557374, 4.96816152, 4.983054790000001, 5.068206870000001]
  tony meo [another player] and me used to spend all of our spare time there   mr white said   we loved the game and the atmosphere||  school went out of the window|| i went for a while and then started taking time off||  mr howard s fellow welshman ray reardon - known by his fellow professionals as  dracula  - won the snooker world championship six times  having left school at to work as a miner|| and terry griffiths  like mr howard from llanelli  won the tournament in it is not known whether the two of them ever clashed cues at jack s||
[1.41421356, 1.73205081, 2.0, 2.0, 2.2118760699999997, 2.21187607, 2.236068, 2.6457512899999998, 2.6457512899999998, 2.99999997, 2.99999997, 3.1622776999999997, 3.1622777, 3.4641015600000005, 3.90006742, 4.22379773, 4.55261084]


' विद्यालय झ्यालबाट गयो || समयको गएँ त्यसपछि लिन थाले|| मिस्टर हावर्डको फेलो वेल्शम्यान रीर्डन - उनका साथी पेशेवरहरू ड्र्याकुला चिनिन्छन् - स्नूकर विश्व च्याम्पियनशिप जितेका वर्षको उमेरमा स्कूल छोडेर खानीको रूपमा गरे|| ल्यानेलीका मिस्टर होवार्ड टेरी ग्रिफिथले सन् प्रतियोगिता जितेका थिए|| दुईबीच ज्याक एसमा कहिले भिडन्त छैन||'